In [90]:
import os
import pandas as pd
import warnings
import funciones_tpm as fc

In [91]:
#Ocultar advertencias/warnings mas no los borra
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [92]:
#Año evaluado
Añoeval = 2025

**Variables globales de las rutas necesarias**

In [93]:
#Rutas del archivo de la COOISPI
Ruta_COOISPI = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\COOISPI\Datos_COOISPI_Mezclas.csv"                          # Ruta COOISPI en csv

#Rutas del archivo del EGE
Ruta_Archivo_EGE = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Backup_BD_Salones\Datos_Mezclas_2021.csv"               # Ruta Mezclas

#Rutas del archivo de Novedades
Ruta_Novedades = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Consolidados_Salones\Mezclas\Mezclas_Novedades_V2.xlsx"   # Ruta del archivo de Novedades

#Rutas del archivo de Consolidados
Ruta_Consolidados = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Consolidados_Salones\Mezclas\Consolidado_V2.xlsx"      # Ruta del archivo de Consolidados

#Ruta de salida
# Ruta_Mezclas = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Consolidados_Salones\Mezclas"                             # DB_Mezclas_Out_vf.csv

#Validacion de existencia de las rutas
for nombre, ruta in {
  "Ruta_COOISPI": Ruta_COOISPI,
  "Ruta_Archivo_EGE": Ruta_Archivo_EGE,
  "Ruta_Novedades": Ruta_Novedades,
  "Ruta_Consolidados": Ruta_Consolidados,
  # "Ruta_Mezclas": Ruta_Mezclas,
}.items():
  if not os.path.exists(ruta):
    raise FileNotFoundError(f"La ruta '{nombre}' no fue encontrada.")
  else:
    print(f"Ruta '{nombre}' encontrada.")

Ruta 'Ruta_COOISPI' encontrada.
Ruta 'Ruta_Archivo_EGE' encontrada.
Ruta 'Ruta_Novedades' encontrada.
Ruta 'Ruta_Consolidados' encontrada.


In [94]:
#Leer_archivo COOISPI csv
df_COOISPI_COMB = fc.leer_archivo(Ruta_COOISPI)

Leyendo archivo...


In [95]:
#Convertir columna 'Codigo' a str para futuros procesor (merge y tabla pivote)
df_COOISPI_COMB['Codigo'] = df_COOISPI_COMB['Codigo'].astype(str)

In [96]:
#Funcion para leer archivo el EGE/TPM
df_Mezclas = fc.leer_archivo(Ruta_Archivo_EGE, 'Datos')

Leyendo archivo...


In [97]:
#Reemplazar el nombre de la columna "Día:" y "Mes_N" para estandarizar los scripts
df_Mezclas.rename(columns={"Día:":"Dia", "Mes_N":"IdMes"}, inplace=True)

In [98]:
#Eliminar_filas_vacias
numeros_vacios_filtrados = fc.eliminar_filas_vacias(df_Mezclas, 'Número')

In [99]:
#Crear_fecha
df_Mezclas = fc.crear_fecha(df_Mezclas)

In [100]:
#Tomaré solo las columnas de interes del informe de mezclas para el sobrepeso
columnas=['Número','Dia','Fecha','Mes:', 'IdMes', 'Año:', 'Máquina / Equipo:', 'Semana:', 'Turno:', 'Código y Descrip. / Producto:', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):','Gramaje (K):']
df_Mezclas = df_Mezclas[columnas]

In [101]:
#Filtrar_desde_anio
df_Mezclas = fc.filtrar_desde_anio(df_Mezclas, 2024)

In [102]:
#Especificar las columnas en las que deseas aplicar el filtro para eliminar los valores nulos de las columnas de Gramaje, Unidades producidas y Peso promedio
columnas_filtrar_nulos = ['Gramaje (K):', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):']

In [103]:
#Eliminar_valores_nulos
df_Mezclas_nan_ceros, df_Mezclas = fc.filtrar_nulos(df_Mezclas, columnas_filtrar_nulos)

**Modificación de los tipos de datos del archivo de TPM de Mezclas**

In [104]:
#Validar_y_convertir_datos
df_non_conver, df_Mezclas = fc.validar_numericos(df_Mezclas, 'Peso Promedio de la unidad (K):')

In [105]:
#Extraer_codigo
df_Mezclas = fc.extraer_codigo(df_Mezclas)

In [106]:
#Convertir a numerico sino asignar NaN
df_Mezclas['Gramaje (K):'] = pd.to_numeric(df_Mezclas['Gramaje (K):'], errors='coerce')

In [107]:
#Eliminar_codigos_nan
df_Mezclas, df_Mezclas_Codigos_nan = fc.eliminar_codigos_nan(df_Mezclas)

In [108]:
#Calcular_columnas
df_Mezclas = fc.calcular_columnas(df_Mezclas)

**Generación de Novedades**

In [109]:
Sobrepeso_Novedades = 0.05 #5% de Sobrepeso

In [110]:
#Generar_novedades
df_Mezclas_Nov_Sobrepeso = fc.generar_novedades(df_Mezclas, Sobrepeso_Novedades)

In [111]:
#Agregar columna 'Costo/kg'
df_Mezclas['Costo/kg'] = 0.0

**Tabla de Consolidado Mensual**

In [112]:
#Pivote_consolidado
df_Mezclas_Mes = fc.pivote_consolidado(df_Mezclas)

In [113]:
#Tomar datos desde el año 2024 en adelante
df_Mezclas_Mes = fc.filtrar_desde_anio(df_Mezclas_Mes, 2024)

**Creación de la agrupación Mensual del archivo de semielaborados**

In [114]:
df_costo_semi = df_COOISPI_COMB.copy(deep=True)

In [115]:
#Creacion de la agrupacion mensual del archivo semielaborado
df_costo_semi_Mes = fc.pivote_semielaborados(df_costo_semi)

**Generación del nuevo archivo consolidado V2**

In [116]:
df_Mezclas_2 = df_Mezclas.copy(deep=True)

In [117]:
#Traer la columna 'Costo/kg' al dataframe original
df_Mezclas_2 = fc.merge_costo(df_Mezclas_2, df_costo_semi_Mes)

In [118]:
#Eliminar la columna Costo/kg antigua y renombrar la nueva
df_Mezclas_2 = fc.modificar_costo(df_Mezclas_2)

In [119]:
#Definir meta
df_Mezclas_2['Meta'] = df_Mezclas_2['Máquina / Equipo:'].map({'ENFLEX': 0.0056,
                                                              'TRANSPACK 2': 0.0033,
                                                              'INGEPACK': 0.0072,
                                                              'ROVEMA':0.0078,
                                                              'LANIC':0.0055,
                                                              'TOYO 7':0.0065,
                                                              'TOYO':0.0088,
                                                              'ENCAPSULADORA':0.006,
                                                              'INGEPACK 2':0.0068})

In [120]:
#Calcular columna 'Ahorros/Perdidas'
df_Mezclas_2 = fc.calcular_ahorros_perdidas(df_Mezclas_2)
# df_Mezclas_2.head()

In [121]:
#Eliminar registro con Número = 81187
df_Mezclas_2.drop(df_Mezclas_2.loc[df_Mezclas_2['Número'] == 81187].index, inplace=True)

**Unión de los dataframes en un dataframe totalizado**

In [122]:
df_totalizado_Mes = df_Mezclas_Mes.copy(deep=True)

In [123]:
#Validar que no hay filas repetidas en df_totalizado_Mes
fc.filas_repetidas(df_totalizado_Mes)

No hay filas duplicadas.


In [124]:
#Fusionar los DataFrames en base a 'Año:', 'Codigo' y 'Mes:'
df_totalizado_Mes = fc.unir_dataframes(df_totalizado_Mes, df_costo_semi_Mes)

In [125]:
#Metodo diferente a los demas calcular_columnas_totalizado ya que sus metas de sobrepeso no son fijas
df_totalizado_Mes = fc.calcular_columnas_totalizado_mezclas(df_totalizado_Mes)
# df_totalizado_Mes.head()

In [126]:
# Obtener Codigos unicos de df_Mezclas
df_Codigos_Mezclas = fc.eliminar_duplicados_columna(df_Mezclas, 'Codigo')

In [127]:
# Obtener Maquinas únicas de TPM o df_Codigos_Mezclas
df_Maquinas_Mezclas = fc.eliminar_duplicados_columna(df_Mezclas, 'Máquina / Equipo:')

In [128]:
# Obtener Codigos únicos de COOISPI
df_Codigos_Mezclas_COOISPI = fc.eliminar_duplicados_columna(df_COOISPI_COMB, 'Codigo')

**Codigos que están en TPM pero no están en la COOISPI**

In [129]:
#Obtener Codigos que están en TPM (df_Codigos_Mezclas) pero NO en COOISPI(df_Codigos_Mezclas_COOISPI)
df_Mezclas_Codigos_unicos = df_Codigos_Mezclas[~df_Codigos_Mezclas['Codigo'].isin(df_Codigos_Mezclas_COOISPI['Codigo'])]

**Generación de un archivo de fechas Automáticas**

In [130]:
#Generar dataframe con fechas dese 2024-01-01 hasta el dia actual
df_Fechas = fc.generar_fechas("2024-01-01")

In [131]:
# print(df_Mezclas.shape)
# print(df_COOISPI_COMB.shape)
# print(df_Mezclas_Mes.shape)
# print(df_Codigos_Mezclas.shape)
# print(df_Maquinas_Mezclas.shape)
# print(df_totalizado_Mes.shape)
# print(df_Fechas.shape)
# print(df_Mezclas_2.shape)
# print()
# print(df_Mezclas_Nov_Sobrepeso.shape)
# print(df_Mezclas_Codigos_nan.shape)
# print(df_Mezclas_Codigos_unicos.shape)
# print(df_Mezclas_nan_ceros.shape)
# print(df_non_conver.shape)

**Consolidado csv**

In [132]:
# Ruta para consolidado df_Mezclas_2
# df_Mezclas_2.to_csv(Ruta_Mezclas + "DB_Mezclas_Out_vf.csv", index=False)

**Archivo de consolidado**

In [133]:
#Diccionario de hojas para el archivo de Consolidado_V2
dataframes_consolidado = {
    "Hoja1": df_Mezclas,
    "Hoja2": df_COOISPI_COMB,
    "Hoja3": df_Mezclas_Mes,
    "Hoja4": df_Codigos_Mezclas,
    "Hoja5": df_Maquinas_Mezclas,
    "Hoja6": df_totalizado_Mes,
    "Hoja7": df_Fechas,
    "Hoja8": df_Mezclas_2,
}
#Actualizar hojas del excel de Consolidado_V2
fc.actualizar_hojas_excel(Ruta_Consolidados, dataframes_consolidado)

Procesando Consolidado...
✅ Las hojas en el Archivo de Consolidado han sido limpiadas y actualizadas correctamente.


**Archivo de novedades: Este archivo compila las diferentes novedades presentes en los archivos de TPM**

In [ ]:
#Diccionario de hojas para el archivo de novedades
dataframes_novedades = {
    "Sobrepeso mayor  5%": df_Mezclas_Nov_Sobrepeso,           #Novedades de sobrepeso
    "Codigos Nulo TPM": df_Mezclas_Codigos_nan,                #Novedades codigos nulos
    "Codigos unicos en TPM": df_Mezclas_Codigos_unicos,        #Codigos presentes es TPM y no en la COOISPI
    "Codigos NA,Vacio, 0": df_Mezclas_nan_ceros,               #Valores nulos o ceros para gramaje, unidades producidas y promedio peso unidad
    "Errores": df_non_conver                                   #Valores no convertibles (NaN)
}
#Guardar el archivo de Novedades
fc.crear_archivo_novedades(Ruta_Novedades, dataframes_novedades)

Procesando Novedades...


✅ El Archivo de Novedades guardado exitosamente con 5 hojas guardadas correctamente.
